#  Advanced CNN

## 저번 시간에 배운 것


저번 시간에 어떻게 CNN 모델을 구성하는 지에 대해 알아봤다. what is convolution, pooling(sampling) and this made convolution layers.. and so on. Activation maps를 만들기 위해 어떤 size의 필터를 만들어야 하는지 결정해야 하는데 어떻게 결정할까?

## Inception modules

Concept: We are going to try all possible holes and combinations.

![inception-modules](img/11-inceptionModule.png)


We are using **1x1 convolution** first and then connect this into different filters(5x5, 3x3, and so on).

### Why 1x1 convolution first?

![1x1-convolution](img/11-1x1convolution.png)

If we use **32** 1x1 filters, then we will have 32 depth convolution maps.

It will save significatantly our computation time.

### How? 

![computation-compare](img/11-computation-compare.png)

Let's say we want to make 32@28x28 feature maps (for example) out of 192@28x28 input. (192 channels)

1. we can use **32** numbers of 5x5 kernel filter (with some paddings). 

2. Otherwise, we can use **16** numbers of 1x1 kernel. and then, use 32  numbers 5x5 kernel filters. 

At first glimpse, first approach seems to look more efficient. But, when we calculate the computation, second approach is much less. That is why we apply 1x1 convolution first and then apply others.


### Implementation 

![inception-module-implementation](img/11-inception-module.png)


In [3]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [4]:
class InceptionA(nn.Module):

    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

In [5]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [9]:
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.293280
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.297219
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.297801
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.294497
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.292915
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.284132
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.298065
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.287186
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.282574
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.273279
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.276375
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.275467
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.258624
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.243830
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.238908
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.174750
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.133663
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.017774
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.700413
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.612088
Train Epoch: 1 [12800/60000 (

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.1977, Accuracy: 9422/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.229629
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.176597
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.110935
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.340516
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.257459
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.272187
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.221395
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.202018
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.073695
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.348464
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.175928
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.167543
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.194731
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.118922
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.204346
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.152082
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.183497
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.188648
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.186076
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.070641
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.039903
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.088042
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.043701
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.060583
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.067792
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.077169
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.131861
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.077283
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.091678
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.041944
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.071396
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.028182
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.044166
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.024644
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.123776
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.074112
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.140844
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.036728
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.077980


Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.083757
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.007671
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.063234
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.044900
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.195113
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.052829
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.062349
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.035308
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.063544
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.153708
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.161109
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.206426
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.049105
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.051080
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.014186
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.041377
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.031976
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.016510
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.072669
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.014891


Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.057161
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.021866
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.040509
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.034754
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.042141
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.060182
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.068460
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.012051
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.283545
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.145739
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.145361
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.017568
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.084120
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.085977
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.163391
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.017826
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.098640
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.007236
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.028675
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.008361


Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.144561
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.021538
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.062204
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.040800
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.087208

Test set: Average loss: 0.0609, Accuracy: 9822/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.011768
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.026507
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.019066
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.138722
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.025685
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.014271
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.025511
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.077920
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.014062
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.013102
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.033120
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.062572
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.008570
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.069729
Train Epoch: 9 [